In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torchtext.data.utils import get_tokenizer
from torch.nn import TransformerEncoder, TransformerEncoderLayer


In [2]:

# first let's do data wrangling

TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

In [3]:
# params and util fn for training
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
# model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
# optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)


In [4]:
# we don't want to bother too much with pos encoding so we just use the class 
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [5]:
# let's train for one batch
ntokens = len(TEXT.vocab.stoi)
data, targets = get_batch(train_data, 0)
# make mask
sz = bptt
mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
# parts for the model
ninp = emsize
pos_encoder = PositionalEncoding(ninp, dropout)
encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
encoder = nn.Embedding(ntokens, ninp)
decoder = nn.Linear(ninp, ntokens)


In [6]:
# run through model
src = data
src1 = encoder(src) * math.sqrt(ninp)
src2 = pos_encoder(src1)
output0 = transformer_encoder(src2, mask)
output = decoder(output0)

In [35]:
print(src[0][0], src1[0][0][:9]) # dict value vs. embedding of value :9 is to effect only, it's really of emsize=200 vec
print(src.size(), src1.size())
print(output.size(), targets.size()   )

tensor(3) tensor([ -0.5931,  14.1556,  11.5772,   7.9735,   5.9866,   5.9855,   8.2021,
        -25.7343, -22.5729], grad_fn=<SliceBackward>)
torch.Size([35, 20]) torch.Size([35, 20, 200])
torch.Size([35, 20, 28785]) torch.Size([700])


In [38]:
loss = criterion(output.view(-1, ntokens), targets)
loss

tensor(10.3970, grad_fn=<NllLossBackward>)

In [39]:
loss.backward()

In [7]:
data

tensor([[    3,    25,  1849,   570,     7,     5,     5,  9258,     4,    56,
             0,     7,     6,  6634,     4,  6603,     6,     5,    65,    30],
        [   12,    66,    13,  4889,   458,     8,  1045,    21, 19094,    34,
           147,     4,     0,    10,  2280,  2294,    58,    35,  2438,  4064],
        [ 3852, 13667,  2962,    68,     6, 28374,    39,   417,     0,  2034,
            29,    88, 27804,   350,     7,    17,  4811,   902,    33,    20],
        [ 3872,     5,     9,     4,   155,     8,  1669,    32,  2634,   257,
             4,     5,     5,    11,  4568,  8205,    78,  5258,  7723, 12009],
        [  884,    91,   963,   294,     4,   548,    29,   279,    37,     4,
           391,    31,     4,  2614,   948, 13583,   405,   545,    15,    16],
        [   12,    25,     5,     5,  1688,     0,    39,    59,  8785,     0,
             6,    13,  3026,    43,    11,     6,     0,   349,  3134,  4538],
        [    3,     6,    82,  1780,    21,   

In [8]:
src1

tensor([[[  6.8754,  -9.4439,  11.5488,  ...,   6.9110,  -8.7892, -27.1390],
         [-15.1194, -12.6068, -22.7539,  ...,  -6.8978,  -8.8547,  -9.9066],
         [ -1.7704, -14.6749,   9.8628,  ...,  -3.7070,  -3.8328, -19.8940],
         ...,
         [  8.4953, -38.1183,  -8.1167,  ...,  -2.1501,  -6.3575, -25.0811],
         [  8.8252,  -3.5102,  12.5300,  ...,  -6.9726,  18.4786,  15.9362],
         [-15.8234,  -3.1877, -16.3614,  ...,  -4.7671,   9.4257,   3.5041]],

        [[  3.6808,  18.5054,  -4.5577,  ..., -21.9113,  -3.3585,  -7.6830],
         [  2.3326, -35.8592,  12.5238,  ...,  -7.0262, -11.6627,  -0.2249],
         [-43.0223, -22.6506,  22.6495,  ...,  -8.3481,  -7.5752,   4.0078],
         ...,
         [  2.9205,   6.9040,   2.5209,  ...,  -2.8129, -16.2818,   3.7259],
         [ -3.8394,   6.7017, -14.7165,  ...,  17.8833, -20.8497,   2.0899],
         [ 18.9062,  10.7315,  24.1147,  ...,  24.6830,  21.8114, -21.2592]],

        [[ 14.7675,   9.5921,  33.8792,  ...

In [9]:
encoder_layers(src1)

tensor([[[ 1.1061, -0.9796,  0.9826,  ...,  1.1279,  0.5106, -1.7847],
         [-1.1990,  0.2679, -1.9748,  ..., -0.5389,  0.2254, -0.5904],
         [ 0.0860, -0.2968, -0.1216,  ...,  0.1845, -0.0208, -0.2638],
         ...,
         [ 0.5171, -2.6012, -0.7301,  ..., -0.7282, -0.4032, -1.4576],
         [ 1.2383,  0.6211,  0.9330,  ..., -0.7069,  1.4402,  0.2654],
         [-1.1168, -0.2211, -0.6438,  ...,  0.3666,  1.4219,  0.2521]],

        [[ 0.3981,  0.4132, -0.1645,  ..., -0.3574,  0.3433, -1.0266],
         [-0.4795, -1.6399,  0.3169,  ..., -0.2813,  0.1042,  0.5923],
         [-2.4172, -1.2791,  2.0421,  ..., -0.2753,  0.7108,  0.2609],
         ...,
         [ 0.0651,  0.8549, -0.5745,  ..., -2.1115, -0.9226,  0.2294],
         [ 0.4881,  0.1432, -1.7524,  ..., -0.0991, -0.8505, -0.1755],
         [ 0.6100,  1.4778,  1.2633,  ...,  2.1395,  1.4001, -1.9058]],

        [[ 1.1407,  1.1470,  2.2177,  ..., -0.0670, -1.4174,  0.7088],
         [ 0.1723,  0.2673,  1.8657,  ...,  1

In [10]:
encoder_layers(src1).size()

torch.Size([35, 20, 200])